In [1]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Rescaling #InputLayer, HiddenLayer, OutputLayer
from tensorflow.keras.models import Sequential #Feed-Forward-NN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt #Plot Images
import os #Access files and folders
import numpy as np #Calculations
from datetime import datetime

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("GPUs detected: ", len(physical_devices))
if physical_devices:
    print("GPU will be used")
    tf.config.experimental.set_memory_growth(physical_devices[0], True) #Use GPU if available

In [ ]:
path = "G:/Python/data/10_animals" #Path to dataset
height = 300; width = 300 #Rescale images in dataset to set size
batch_size = 64 #determine batch size for training

train = tf.keras.preprocessing.image_dataset_from_directory( #Loading training-data
    path,
    image_size=(height, width),
    validation_split=0.25, #Determine how many images will be used for validation
    subset="training",
    seed=123, #Shuffling and Transformation
    batch_size=batch_size
)

val = tf.keras.preprocessing.image_dataset_from_directory( #Loading validation-data
    path,
    image_size=(height, width),
    validation_split=0.25,
    subset="validation",
    seed=123,
    batch_size=batch_size
)

classes = train.class_names

In [ ]:
model = Sequential([
    InputLayer((height, width, 3)), #InputLayer
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),

    Conv2D(32, (3,3), padding='same', activation='relu'), #ConvolutionLayer
    MaxPooling2D(), #PoolingLayer

    Conv2D(64, (3,3), padding='same', activation='relu'), #ConvolutionLayer
    MaxPooling2D(), #PoolingLayer

    Conv2D(128, (3,3), padding='same', activation='relu'), #ConvolutionLayer
    MaxPooling2D(), #PoolingLayer

    Conv2D(128, (3,3), padding='same', activation='relu'), #ConvolutionLayer
    MaxPooling2D(), #PoolingLayer

    Flatten(), #FlattenLayer transforms input into 1-dimensional Array to be processed by Dense Layers
    Dense(128, activation='relu'), #FullyConnectedLayer
    Dense(64, activation='relu'), #FullyConnectedLayer
    Dense(len(classes), activation='softmax') #OutputLayer
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'] #tracking for accuracy
)

model.summary() #overview of built model

In [ ]:
model.fit(train, validation_data=val, epochs=20) #Train the model

In [7]:
model.save(os.path.join("C:/Users/maxis/Desktop/Proseminar - CNN/Models","animals_10_(overfitting_cut)_59_val_accuracy.h5"))

In [ ]:
img = tf.keras.preprocessing.image.load_img("G:/Python/data/img_from_google/cat1.jpg", target_size=(height, width)) #Load image thats not already in the dataset

plt.imshow(img) #Plot image
plt.show()

img = tf.keras.preprocessing.image.img_to_array(img) #Convert image into array
img = np.expand_dims(img, axis=0) #Add dimensions to array to be processed by the model

prediction = model.predict(img) #Model predicts class of image

print()

prediction_probabilities = tf.math.top_k(prediction, k=5) #Get top-5 predictions
top_5_scores = prediction_probabilities.values.numpy().tolist() #Get top-5 probabilities
dict_class_entries = prediction_probabilities.indices.numpy().tolist() #Get matching classes to probabilities
for label, confidence in zip(dict_class_entries[0], top_5_scores[0]):
    print(f"Image predicted to be {classes[label]} with confidence of {round(confidence*100, 3)}%") #Printing the class and probability